In [133]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [43]:
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/stroke_/test.csv')

display(train.head())
display(test.head())

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1192,Female,31,0,0,No,Govt_job,Rural,70.66,27.2,never smoked,0
1,77,Female,13,0,0,No,children,Rural,85.81,18.6,Unknown,0
2,59200,Male,18,0,0,No,Private,Urban,60.56,33.0,never smoked,0
3,24905,Female,65,0,0,Yes,Private,Urban,205.77,46.0,formerly smoked,1
4,24257,Male,4,0,0,No,children,Rural,90.42,16.2,Unknown,0


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,47472,Female,58,0,0,Yes,Private,Urban,107.26,38.6,formerly smoked
1,36841,Male,78,1,0,Yes,Self-employed,Rural,56.11,25.5,formerly smoked
2,3135,Female,73,0,0,No,Self-employed,Rural,69.35,NaN,never smoked
3,65218,Male,2,0,0,No,children,Rural,109.10,20.0,Unknown
4,1847,Female,20,0,0,No,Govt_job,Rural,79.53,NaN,never smoked


# EDA

In [44]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1137 non-null   int64  
 1   gender             1137 non-null   object 
 2   age                1137 non-null   object 
 3   hypertension       1137 non-null   int64  
 4   heart_disease      1137 non-null   int64  
 5   ever_married       1137 non-null   object 
 6   work_type          1137 non-null   object 
 7   Residence_type     1137 non-null   object 
 8   avg_glucose_level  1137 non-null   float64
 9   bmi                1085 non-null   float64
 10  smoking_status     1137 non-null   object 
 11  stroke             1137 non-null   int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 106.7+ KB


In [45]:
display(test.isnull().sum())
train.isnull().sum()

id                    0
gender                0
age                   0
hypertension          0
heart_disease         0
ever_married          0
work_type             0
Residence_type        0
avg_glucose_level     0
bmi                  57
smoking_status        0
dtype: int64

id                    0
gender                0
age                   0
hypertension          0
heart_disease         0
ever_married          0
work_type             0
Residence_type        0
avg_glucose_level     0
bmi                  52
smoking_status        0
stroke                0
dtype: int64

In [46]:
display(train.describe())
test.describe()

,id,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,1137.000000,1137.000000,1137.000000,1137.000000,1085.000000,1137.000000
mean,36750.933157,0.118734,0.068602,107.664002,29.198065,0.120493
std,21112.281253,0.323617,0.252887,47.618723,7.669615,0.325680
min,77.000000,0.000000,0.000000,55.270000,11.300000,0.000000
25%,17986.000000,0.000000,0.000000,77.600000,24.100000,0.000000
50%,37479.000000,0.000000,0.000000,91.820000,28.500000,0.000000
75%,55410.000000,0.000000,0.000000,113.850000,33.200000,0.000000
max,72918.000000,1.000000,1.000000,266.590000,64.400000,1.000000


,id,age,hypertension,heart_disease,avg_glucose_level,bmi
count,932.000000,932.000000,932.000000,932.000000,932.000000,875.000000
mean,36644.275751,44.821888,0.114807,0.050429,107.130708,28.644114
std,21087.289033,22.753582,0.318960,0.218946,45.336215,7.926094
min,121.000000,0.000000,0.000000,0.000000,55.230000,12.800000
25%,17584.750000,26.000000,0.000000,0.000000,77.820000,23.450000
50%,37056.500000,47.000000,0.000000,0.000000,92.840000,27.700000
75%,54125.500000,63.000000,0.000000,0.000000,115.560000,32.300000
max,72911.000000,82.000000,1.000000,1.000000,271.740000,92.000000


In [47]:
train.select_dtypes(include='object')

,gender,age,ever_married,work_type,Residence_type,smoking_status
0,Female,31,No,Govt_job,Rural,never smoked
1,Female,13,No,children,Rural,Unknown
2,Male,18,No,Private,Urban,never smoked
3,Female,65,Yes,Private,Urban,formerly smoked
4,Male,4,No,children,Rural,Unknown
...,...,...,...,...,...,...
1132,Female,1,No,children,Rural,Unknown
1133,Male,6,No,children,Rural,Unknown
1134,Male,81,Yes,Private,Urban,formerly smoked
1135,Female,81,Yes,Private,Rural,never smoked


In [48]:
train.smoking_status.value_counts()

never smoked       416
Unknown            352
formerly smoked    205
smokes             164
Name: smoking_status, dtype: int64

In [49]:
test.smoking_status.value_counts()

never smoked       323
Unknown            275
formerly smoked    169
smokes             165
Name: smoking_status, dtype: int64

# 카테고리 컬럼 전처리

In [50]:
train.loc[train['age']=='*82', 'age'] = 82

In [51]:
train.age = train.age.astype(int)

In [52]:
train.drop(train[train.gender=='Other'].index, axis=0, inplace=True)

In [53]:
# 카테고리 컬럼 원핫인코딩

train = pd.get_dummies(train, drop_first=True)
test = pd.get_dummies(test, drop_first=True)

In [54]:
y = train['stroke']

x = train.drop(columns=['id', 'stroke'])
test.drop(columns='id', inplace=True)

# 결측치 처리

In [55]:
x[x.bmi.isnull()].index
x[x.bmi.notnull()].index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            1126, 1127, 1128, 1129, 1130, 1132, 1133, 1134, 1135, 1136],
           dtype='int64', length=1084)

In [56]:
dumb = pd.concat([x, test])
dumb

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,31,0,0,70.66,27.2,0,0,0,0,0,0,0,0,1,0
1,13,0,0,85.81,18.6,0,0,0,0,0,1,0,0,0,0
2,18,0,0,60.56,33.0,1,0,0,1,0,0,1,0,1,0
3,65,0,0,205.77,46.0,0,1,0,1,0,0,1,1,0,0
4,4,0,0,90.42,16.2,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
927,58,0,0,149.75,27.0,0,1,0,1,0,0,1,0,0,0
928,34,0,0,89.68,23.2,1,0,0,1,0,0,0,0,0,1
929,46,0,0,92.81,30.8,1,1,0,1,0,0,0,0,0,0
930,72,1,0,221.79,30.0,0,1,0,1,0,0,1,0,1,0


In [57]:
rfr = RandomForestRegressor()

In [58]:
rfr.fit(dumb[dumb.bmi.notnull()].drop(columns=['bmi']), dumb[dumb.bmi.notnull()].bmi)

RandomForestRegressor()

In [59]:
predict_bmi = rfr.predict(dumb[dumb.bmi.isnull()].drop(columns=['bmi']))
predict_bmi

In [61]:
predict_bmi

array([32.713, 36.276, 37.214, 30.141, 23.399, 34.351, 34.509, 27.995,
       36.741, 31.074, 32.546, 36.508, 28.267, 28.133, 31.64 , 29.318,
       33.897, 28.139, 29.259, 28.304, 29.882, 33.975, 25.576, 18.278,
       29.017, 28.846, 30.659, 28.5  , 26.389, 23.556, 31.145, 33.116,
       33.127, 32.36 , 33.963, 38.631, 23.949, 31.152, 31.635, 31.46 ,
       30.014, 28.274, 34.208, 28.375, 29.604, 29.207, 36.847, 35.18 ,
       27.052, 20.421, 35.309, 30.282, 32.91 , 26.56 , 30.265, 47.946,
       28.177, 26.744, 32.754, 35.932, 18.375, 29.976, 36.687, 33.334,
       17.58 , 33.164, 31.534, 33.048, 26.633, 20.094, 29.399, 30.164,
       32.67 , 29.239, 25.844, 28.523, 27.528, 27.838, 31.477, 35.146,
       26.933, 31.334, 40.608, 29.212, 29.099, 31.998, 33.165, 31.821,
       34.914, 28.871, 34.109, 23.574, 33.63 , 33.812, 29.268, 36.719,
       31.972, 42.739, 35.535, 22.411, 29.568, 35.742, 36.636, 34.79 ,
       33.422, 35.339, 38.69 , 31.398, 27.453])

In [62]:
dumb.loc[dumb.bmi.isnull(), 'bmi'] = predict_bmi

In [63]:
dumb.isnull().sum()

age                               0
hypertension                      0
heart_disease                     0
avg_glucose_level                 0
bmi                               0
gender_Male                       0
ever_married_Yes                  0
work_type_Never_worked            0
work_type_Private                 0
work_type_Self-employed           0
work_type_children                0
Residence_type_Urban              0
smoking_status_formerly smoked    0
smoking_status_never smoked       0
smoking_status_smokes             0
dtype: int64

In [64]:
x = dumb[:len(x)]
test = dumb[len(x):]

# 수치형 변수 처리 (스탠다드 스케일링)

In [70]:
numeric_columns = ['avg_glucose_level', 'bmi']
test.describe()

,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Male,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
count,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000,932.000000
mean,44.821888,0.114807,0.050429,107.130708,28.806763,0.400215,0.688841,0.002146,0.578326,0.160944,0.127682,0.498927,0.181330,0.346567,0.177039
std,22.753582,0.318960,0.218946,45.336215,7.818397,0.490205,0.463216,0.046299,0.494092,0.367677,0.333915,0.500267,0.385499,0.476132,0.381906
min,0.000000,0.000000,0.000000,55.230000,12.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,26.000000,0.000000,0.000000,77.820000,23.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,47.000000,0.000000,0.000000,92.840000,27.869000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,63.000000,0.000000,0.000000,115.560000,32.500000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,82.000000,1.000000,1.000000,271.740000,92.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [73]:
scaler = StandardScaler()

In [74]:
x[numeric_columns] = scaler.fit_transform(x[numeric_columns])
test[numeric_columns] = scaler.transform(test[numeric_columns])

C:\Users\lovej\AppData\Local\Temp\ipykernel_23696\1678942027.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[numeric_columns] = scaler.fit_transform(x[numeric_columns])
C:\Users\lovej\AppData\Local\Temp\ipykernel_23696\1678942027.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[numeric_columns] = scaler.transform(test[numeric_columns])


# 모델링

In [98]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

In [129]:
rfc = RandomForestClassifier(random_state=42, max_depth=6)

In [130]:
rfc.fit(x_train, y_train)

RandomForestClassifier(max_depth=6, random_state=42)

In [131]:
predict_rfc = rfc.predict(x_val)

In [132]:
roc_auc_score(predict_rfc, y_val)

0.6914893617021277

In [146]:
xgb = XGBClassifier(random_state=42, max_depth=6)

In [147]:
xgb.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=42, ...)

In [148]:
predict_xgb = xgb.predict(x_val)

In [149]:
roc_auc_score(predict_xgb, y_val)

0.6457960644007156

# 예측

In [169]:
rfc_final = RandomForestClassifier(random_state=42, n_estimators=400)

In [170]:
rfc_final.fit(x, y)

RandomForestClassifier(n_estimators=400, random_state=42)

In [171]:
predict_final = rfc_final.predict(test)

In [172]:
predict_final

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [173]:
submission = pd.DataFrame()
submission['pred'] = predict_final
submission.value_counts()

pred
0       917
1        15
dtype: int64

In [175]:
# submission.to_csv("007001671.csv", index=False)

,pred
0,0
1,0
2,0
3,0
4,0
...,...
927,0
928,0
929,0
930,0
